<div style="font-size:28pt; line-height:30pt; font-weight:bold; text-align:center;">Large Margin Deep Networks for Classification </div>

The large margin principle has produced remarkable theoretical and empirical results for classification (SVM - Vapnik, 1995) and regression problems (SVR - Drucker et al., 1997).

Desirable benefits of large margin classifiers include : 
- better generalization properties
- robustness to input perturbations.
  
However, exact large margin algorithms are only suitable for **shallow models** where the margin has an analytical form (the l2 norm of the parameters). To overcome the limitations of classical margin approaches, G.F. Elsayed et al. designed a novel **loss function based on a first-order approximation of the margin**. 

This loss function is applicable to any network architecture (e.g., arbitrary depth, activation function, use of convolutions, residual networks), and complements existing general-purpose regularization techniques such as weight-decay, dropout and batch normalization.

## **I. Theoretical definition**

#### **1. Large margin principle**
*The following definitions have already been explored in the SVM course in AML.*

Consider a classification problem with n classes.  
Suppose we use a function $f_i: X → R$, for $i = 1,. . ., n$ that generates a prediction score for classifying the input vector $x \in X$ to class $i$. The predicted label is decided by the class with maximal score, i.e. $i^∗ = \operatorname{argmax}_i f_i(x)$.

Define the decision boundary for each class pair {i, j} as:
$$D_{i,j} = \{x | f_i(x) = f_j (x)\} \;\;\;\;(1) $$  

Under this definition, the distance of a point x to the decision boundary $D_{i,j}$ is defined as the smallest distance to be moved so that x
reaches the decision boundary, implying a score tie. Hence,
$$d_{f,x,\{i,j\}} = \min_δ \|δ\|_p \; \; s.t. \; \; f_i(x + δ) = f_j (x + δ) \;\;\;\;(2) $$

#### **2. Large Margin Deep Networks**

Using the above distance, we can develop a large margin loss:

We start with a training set consisting of pairs $(x_k, y_k)$. We penalize the displacement of each $x_k$ to satisfy the margin constraint for separating class $y_k$ from class i ($i \ne y_k$). This implies using the following loss function:
$$ \max\{0, γ + d_{f,x_k,\{i,y_k\}} sign (f_i(x_k) − f_{yk}(x_k))\} \;\;\;\;(3) $$
 
In a multiclass setting, we aggregate individual losses arising from each $i \ne y_k$ by some aggregation operator *A* ($max$ or $\sum$):

$$A_{i \ne y_k} \max\{0, γ + d_{f,x_k,\{i,y_k\}} sign (f_i(x_k) − f_{yk}(x_k))\}  \;\;\;\;(4) $$

In order to learn $f_i$, we assume it is parameterized by a vector w and should use the notation $f_i(x; w)$; for brevity we keep using the notation $f_i(x)$. The goal is to minimize the loss w.r.t. w:
$$w^∗  = \operatorname{argmin}_w \sum_k A_{i \ne y_k} \max\{0, γ + d_{f,x_k,\{i,y_k\}} sign (f_i(x_k) − f_{yk}(x_k))\} \;\;\;\;(5) $$

The above formulation depends on d, whose exact computation from (2) is intractable when $f_i$’s are nonlinear (remember we are dealing with neural networks!). Instead, we present an **approximation to d**:
$$ \tilde{d}_{f,x,\{i,j\}} = \frac{|f_i(x) − f_j (x)|}{\| ∇_xf_i(x) - ∇_xf_j(x) \|_q} \;\;(6)$$
where $\|\|_q$ is the dual-norm of $\|\|_p$. Using the linear approximation, the loss function becomes after simplification:
$$ \hat{w} =  \operatorname{argmin}_w \sum_k A_{i \ne y_k} \max \left\{0, γ + \frac{f_i(x_k) − f_{y_k} (x_k)}{\| ∇_xf_i(x_k) - ∇_xf_{y_k}(x_k) \|_q} \right\} \;\; (7)$$

*NB.* (6) coincides with an SVM for the special case of a linear classifier.

#### **2. Margin for Hidden Layers**
The classic notion of margin is defined based on the distance of input samples from the decision boundary using input/output association. In deep networks, however, the output is shaped from input by going through a number of transformations (layers). Thus, **we can define the margin based on any intermediate representation and the ultimate decision boundary**.

The input x in the margin formulation (7) is replaced with the intermediate representation of x. More precisely, let $h_l$ denote the output of the l’th layer ($h_0 = x$) and $γ_l$ be the margin enforced for its corresponding representation. Then the margin loss (7) can be adapted as following :
$$ \hat{w} =  \operatorname{argmin}_w \sum_{l,k} A_{i \ne y_k} \max \left\{0, γ_l + \frac{f_i(x_k) − f_{y_k} (x_k)}{\| ∇_{h_l}f_i(x_k) - ∇_{h_l}f_{y_k}(x_k) \|_q} \right\} \;\; (8)$$

## **II. Experiments**

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils import data
from torchvision import datasets
from torchvision import transforms

from torch.optim import Adam

from LargeMarginLoss import LargeMarginLoss
from test import test
from train import train_ce, train_lm
from network import Net

#### **1. Data loading**
We use the MNIST dataset.

In [2]:
train_loader = data.DataLoader(
        datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
        batch_size=256, shuffle=True, drop_last=True)

test_loader = data.DataLoader(
        datasets.MNIST('./data', train=False,
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
        batch_size=2048, shuffle=False, drop_last=False)

#### **2. Network training with the Large Margin loss.**

In [3]:
lm = LargeMarginLoss(
    gamma=5,
    top_k=3,
    dist_norm=np.inf
)

device = torch.device("cpu")

net = Net().to(device)
optim = Adam(net.parameters())
for i in range(0, 5):
    train_lm(net, train_loader, optim, i, lm, device)
    test(net, test_loader, device)

Train Epoch: 0 [0/60000 (0%)]	Loss: 5.024226
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.817788
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.611039
Test set: Accuracy: 9782/10000 (98%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.283682
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.473816
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.438349
Test set: Accuracy: 9864/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.337239
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.795408
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.331209
Test set: Accuracy: 9885/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.080955
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.082739
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.178787
Test set: Accuracy: 9900/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.176652
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.890306
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.350686
Test set: Accuracy: 9900/10000 (99%)



#### **2. Network training with the Cross Entropy loss.**
Comparing with Cross Entropy.

In [4]:
net = Net().to(device)
optim = Adam(net.parameters())
for i in range(0, 5):    
    train_ce(net, train_loader, optim, i, device)
    test(net, test_loader, device)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.299823
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.027824
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.091731
Test set: Accuracy: 9795/10000 (98%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.033656
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.057787
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.028668
Test set: Accuracy: 9870/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.031245
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.028349
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.024944
Test set: Accuracy: 9862/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.023172
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.018221
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.033606
Test set: Accuracy: 9912/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.029208
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.018643
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.004380
Test set: Accuracy: 9917/10000 (99%)



Cross Entropy and Large Margin loss seem to lead to similar accuracies.

#### **4. Network training with SVM.**
We compare results to the SVM results.

In [5]:
X_train = []
y_train = []
for (data, target) in train_loader:
    X_train.append(torch.reshape(data,(256,-1)))
    y_train.append(torch.reshape(target,(256,-1)))

X_train = np.concatenate(X_train,axis=0)
y_train = np.concatenate(y_train,axis=0)
    
X_test = []
y_test = []
for (data, target) in test_loader:
    X_test.append(torch.reshape(data,(-1,784)))
    y_test.append(torch.reshape(target,(-1,1)))

X_test = np.concatenate(X_test,axis=0)
y_test = np.concatenate(y_test,axis=0)

In [32]:
from sklearn import svm
from sklearn.metrics import accuracy_score

clf = svm.SVC(kernel='rbf', gamma=10)
clf.fit(X_train[:10000,:], y_train[:10000])

y_test_pred = clf.predict(X_test)
accuracy_score(y_test, y_test_pred)

0.1135

<div class="alert alert-success"><b>Things to keep in mind:</b>

- **Decision Boundary**: Separation point between two different classes. At the decision boundary, there is ambiguity in class decisions.
- **Margin**: The smallest non negative distance between decision boundary and closest class point
- **Support Vector machines**: The most well known maximum margin principle based classification models - use support vectors (points closest to
decision boundary) to estimate margin.
- **Margins in Deep Networks**: Easy to compute in output space, very difficult (sometimes impossible) to compute in input space. The solution is a novel network-agnostic loss function which captures the principle of large margin separation in both the input and hidden layers for deep neural networks.
</div>